In [66]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [67]:
data = pd.read_csv('./data/retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [68]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

# Оценивание
За выполнени каждого задания 1 балл

4 балла -> отл

3 балла -> хор

И тд

In [69]:
users, items, interactions = data.user_id.nunique(), data.item_id.nunique(), data.shape[0]

print('# users: ', users)
print('# items: ', items)
print('# interactions: ', interactions)

# users:  2499
# items:  89051
# interactions:  2396804


In [70]:
# Группируем товары по количеству купленных
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()

# Переименовываем столбец в количество проданных
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

# Смотрим на верхние 5 элементов
popularity.head(5)

,item_id,n_sold
0,25671,6
1,26081,1
2,26093,1
3,26190,1
4,26355,2


In [71]:
# Получаем список топ 5000 продаваемых идентификаторов товаров
top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000)['item_id'].tolist()

In [72]:
# Выбираем в качестве data_train товары из топ 5000
data_train = data_train.loc[data_train['item_id'].isin(top_5000), :]

### Задание 0. Товар 999999
На вебинаре мы использовали товар 999999 - что это за товар?

**Ответ:** прочие товары (принимаем за купленные товары те, которые пользователь не купил из топа)

Зачем он нужен?

**Ответ:** чтобы не терять пользователей, не купивших ни одного популярного товара из выборки

Используя этот товар мы смещаем качество рекомендаций.
В какую сторону?

**Ответ:** в сторону непопулярных товаров

Можно ли удалить этот товар?   

**Ответ:** удалить конечно можем, но качество обучения пострадает

Уберите этот товар и сравните с качеством на семинаре.

In [73]:
# Заведем фиктивный item_id (если юзер НЕ покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 
user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат sparse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

In [74]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [75]:
%%time

model = ItemItemRecommender(K=5, num_threads=4)
model.fit(csr_matrix(user_item_matrix).T.tocsr(), show_progress=True)
recs = model.recommend(userid=userid_to_id[2],
                        user_items=csr_matrix(user_item_matrix).tocsr(),
                        N=5, 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=True)

100%|██████████| 5000/5000 [00:00<00:00, 13377.09it/s]
Wall time: 987 ms


In [76]:
recs

[(3407, 70158.0),
 (2148, 57211.0),
 (3586, 27050.0),
 (3946, 19897.0),
 (2307, 18127.0)]

In [77]:
result = pd.read_csv('data/predictions_basic.csv')
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[12695716, 8068934, 12428458, 434997, 858425]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[13910611, 911878, 12301378, 9270394, 1109089]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[999999, 1082185, 1098066, 6534178, 1127831]"


In [106]:
%%time
# Добавляем результат ItemItemRecommender
result['itemitem_only_5000'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in recs])

Wall time: 3.99 ms


### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. вес = log(sales_sum товара)
- Придумайте пример 3 весов, посчитайте weighted_random_recommendation для разных весов

In [78]:
data.groupby(by='item_id')['sales_value'].sum().reset_index()

,item_id,sales_value
0,25671,20.94
1,26081,0.99
2,26093,1.59
3,26190,1.54
4,26355,1.98
...,...,...
89046,17991689,2.49
89047,17991691,2.49
89048,18000012,19.96
89049,18024155,3.99


In [79]:
def get_log_item_weights(data):
    item_weights = data.groupby(by='item_id')['sales_value'].sum().reset_index().rename(columns={'sales_value':'weight'})
    item_weights['weight'] += 1
    item_weights['weight'] = np.log(item_weights['weight'])
    item_weights['weight'] /= item_weights['weight'].sum()
    
    return item_weights

log_item_weights = get_log_item_weights(data)

In [80]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    items = np.array(items_weights['item_id'])
    recs = np.random.choice(items, size=n, replace=False, p=items_weights['weight'])
    
    return recs.tolist()

Сделайте предсказания

In [81]:
%%time

# your_code
result['weighted_log_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(log_item_weights))
result.head(2)

Wall time: 2.39 s


,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases,weighted_log_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[12695716, 8068934, 12428458, 434997, 858425]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[999999, 1082185, 1029743, 995785, 1004906]","[13511451, 8019534, 910373, 9337530, 9802787]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[13910611, 911878, 12301378, 9270394, 1109089]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[999999, 1082185, 1098066, 6534178, 1127831]","[12814902, 1106445, 1108940, 1063899, 1154812]"


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма (с вебинара и weighted_random_recommendation) с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество? Почему?

In [82]:
# result = pd.read_csv('./data/predictions_basic.csv') # загрузка predict с семианара
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases,weighted_log_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[12695716, 8068934, 12428458, 434997, 858425]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[999999, 1082185, 1029743, 995785, 1004906]","[13511451, 8019534, 910373, 9337530, 9802787]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[13910611, 911878, 12301378, 9270394, 1109089]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[999999, 1082185, 1098066, 6534178, 1127831]","[12814902, 1106445, 1108940, 1063899, 1154812]"


In [94]:
def precision_at_k(recommended_list, bought_list, k=8):
    if type(bought_list) == str:
        bought_str_list = bought_list[1:-1].split(', ')
        bought_list = [int(item) for item in bought_str_list]

    if type(recommended_list) == str:
        bought_str_list = recommended_list[1:-1].split(', ')
        recommended_list = [int(item) for item in bought_str_list]

    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    flags = np.isin(bought_list, recommended_list)
    precision = flags.sum() / len(recommended_list)

    return precision

In [95]:
?map

Init signature: map(self, /, *args, **kwargs)
Docstring:     
map(func, *iterables) --> map object

Make an iterator that computes the function using arguments from
each of the iterables.  Stops when the shortest iterable is exhausted.
Type:           type
Subclasses:     


In [101]:
models_list = [model_ for model_ in list(result)[2:] ]

In [103]:
pd.DataFrame(
    [
        (model, round(result.apply(lambda row: precision_at_k(recommended_list=row[model], bought_list=row['actual']), axis=1).mean(), 5)) for model in models_list
    ], 
    columns=['model name', 'precision@k']
)


,model name,precision@k
0,random_recommendation,0.00078
1,popular_recommendation,0.15524
2,itemitem,0.13692
3,cosine,0.13291
4,tfidf,0.13898
5,own_purchases,0.17969
6,weighted_log_random_recommendation,0.00098


### Лучше всего отрабатывают популярные рекомендации и рекомендации на основе уже сделанных покупок

### Задание 3. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.
- Попробуйте стратегии ансамблирования изученных алгоритмов


In [107]:

result.apply(lambda row: precision_at_k(row['itemitem_only_5000'], row['actual'], k=4), axis=1).mean()

0.14409892262487758

In [109]:
# Подберем коэффициенты:

for k in range(1, 20):
    model = ItemItemRecommender(K=k, num_threads=4)                            # k - ближайшие соседи
    model.fit(csr_matrix(user_item_matrix).T.tocsr(), show_progress=True)      
    recs = model.recommend(userid=userid_to_id[2],                             
                            user_items=csr_matrix(user_item_matrix).tocsr(),  
                            N=5,                                               # кол-во рекомендаций 
                            filter_already_liked_items=False, 
                            filter_items=None, 
                            recalculate_user=True)
    result[f'itemitem_K={k}'] = result['user_id'].apply(lambda x: [id_to_itemid[rec[0]] for rec in recs])

100%|██████████| 5000/5000 [00:00<00:00, 12729.92it/s]


In [111]:
pd.DataFrame(
    [
        (model_, round(result.apply(lambda row: precision_at_k(recommended_list=row[str(model_)], bought_list=row['actual']), axis=1).mean(), 5)) for model_ in list(result)[2:]
    ], 
    columns=['model name', 'precision@k']
)

,model name,precision@k
0,random_recommendation,0.00078
1,popular_recommendation,0.15524
2,itemitem,0.13692
3,cosine,0.13291
4,tfidf,0.13898
5,own_purchases,0.17969
6,weighted_log_random_recommendation,0.00098
7,itemitem_only_5000,0.14574
8,itemitem_K=1,0.12047
9,itemitem_K=2,0.14662


### Можно остановиться на K = 7, удалив остальные

In [117]:

columns_to_drop = ['itemitem_K=' + str(num) for num in [1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]]

In [121]:
result.drop(columns=columns_to_drop, inplace=True)


pd.DataFrame(
    [
        (model_, round(result.apply(lambda row: precision_at_k(recommended_list=row[str(model_)], bought_list=row['actual']), axis=1).mean(), 5)) for model_ in list(result)[2:]
    ], 
    columns=['model name', 'precision@k']
)

,model name,precision@k
0,random_recommendation,0.00078
1,popular_recommendation,0.15524
2,itemitem,0.13692
3,cosine,0.13291
4,tfidf,0.13898
5,own_purchases,0.17969
6,weighted_log_random_recommendation,0.00098
7,itemitem_only_5000,0.14574
8,itemitem_K=7,0.16357


### Задание 4. Улучшение детерминированных алгоритмов
На семинаре мы рассматривали 



Далее $U \equiv N_i(u) $

$$r_{u,i} =  \frac{1}{S}\sum\limits_{v \in U}\operatorname{sim}(u,v)r_{v, i}$$
$$ S = \sum\limits_{v \in U} \operatorname{sim}(u,v)$$

Предлагается улучшить эту формулу и учесть средние предпочтения всех пользователей

$$r_{u,i} = \mu + \bar{r_u} + \frac{1}{S}\sum\limits_{v \in U}\operatorname{sim}(u,v)(r_{v, i}-\bar{r_{v}} - \mu)$$

Какие смысл имееют $ \mu $ и $ \bar{r_u}$ ?

Реализуйте алгоритм, прогнозирующий рейтинги на основе данной формулы, на numpy (векторизованно!)

В качестве схожести возьмите CosineSimilarity.

Примените к user_item_matrix. В качестве рейтингов возьмите количество или стоимость купленного товара. 
Данный алгоритм предсказывает рейтинги. Как на основании предсказанных рейтингов предсказать факт покупки?

Предложите вариант.
Посчитайте accuracy@5 и сравните с алгоритмами, разобранными на вебинаре.

In [122]:
from sklearn.metrics.pairwise import cosine_similarity

In [123]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0)

In [125]:
mu = user_item_matrix.mean().mean()         # ОТВЕТ НА ВОПРОС: μ - Средняя оценка всех товаров
ru = user_item_matrix.mean(axis=1) # ОТВЕТ НА ВОПРОС: r_u - Средняя оценка данного пользователя

In [128]:
cosine_matrix = (user_item_matrix.T - ru_overline.values - mu).T
cosine_matrix

item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,-0.27655,-0.27655,-0.27655,-0.27655,-0.27655,-0.27655,-0.27655,-0.27655,-0.27655,-0.27655,...,-0.27655,-0.27655,-0.27655,-0.27655,0.72345,-0.27655,-0.27655,-0.27655,-0.27655,-0.27655
2,-0.19655,-0.19655,-0.19655,-0.19655,-0.19655,-0.19655,-0.19655,-0.19655,-0.19655,-0.19655,...,-0.19655,-0.19655,-0.19655,-0.19655,-0.19655,-0.19655,-0.19655,-0.19655,-0.19655,-0.19655
3,-0.25775,-0.25775,-0.25775,-0.25775,-0.25775,-0.25775,-0.25775,-0.25775,-0.25775,-0.25775,...,-0.25775,-0.25775,-0.25775,-0.25775,-0.25775,-0.25775,-0.25775,-0.25775,-0.25775,-0.25775
4,-0.15115,-0.15115,-0.15115,-0.15115,-0.15115,-0.15115,-0.15115,-0.15115,-0.15115,-0.15115,...,-0.15115,-0.15115,-0.15115,-0.15115,-0.15115,-0.15115,-0.15115,-0.15115,-0.15115,-0.15115
5,-0.13275,-0.13275,-0.13275,-0.13275,-0.13275,-0.13275,-0.13275,-0.13275,-0.13275,-0.13275,...,-0.13275,-0.13275,-0.13275,-0.13275,-0.13275,-0.13275,-0.13275,-0.13275,-0.13275,-0.13275
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2496,-0.31595,-0.31595,-0.31595,-0.31595,-0.31595,-0.31595,-0.31595,-0.31595,-0.31595,-0.31595,...,-0.31595,-0.31595,-0.31595,-0.31595,-0.31595,-0.31595,-0.31595,-0.31595,-0.31595,-0.31595
2497,-0.37815,-0.37815,-0.37815,-0.37815,-0.37815,-0.37815,-0.37815,-0.37815,-0.37815,-0.37815,...,-0.37815,0.62185,-0.37815,-0.37815,-0.37815,-0.37815,-0.37815,-0.37815,-0.37815,-0.37815
2498,1.79105,-0.20895,-0.20895,-0.20895,-0.20895,-0.20895,-0.20895,-0.20895,-0.20895,-0.20895,...,-0.20895,-0.20895,-0.20895,-0.20895,-0.20895,-0.20895,-0.20895,-0.20895,-0.20895,-0.20895


In [ ]:
model = ItemItemRecommender(K=k, num_threads=4)
model.fit(csr_matrix(user_item_matrix).T.tocsr(), show_progress=True)      
recs = model.recommend(userid=userid_to_id[2],                             
                       user_items=csr_matrix(user_item_matrix).tocsr(),  
                       N=5,
                       filter_already_liked_items=False, 
                       filter_items=None, 
                       recalculate_user=False)